<a href="https://colab.research.google.com/github/Mauregina/Mauregina/blob/main/CapitalRegions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project understanding

| region_id | name               | start_date | end_date | wikidata
|-----------|--------------------|------------|----------|----------|
| 1619      | Monastic State of the Order of Malta   | 1530       |1798      | Q97464297
| 1628      | Republic of Malta  | 1974       |9999      | Q233
| 1629      | Colonial Brazil    | 1530       |1580      | Q2088324
| 1630      | Colonial Brazil    | 1580       |1640      | Q2088324
| 1631      | Colonial Brazil    | 1640       |1714      | Q2088324
| 1632      | Colonial Brazil    | 1714       |1815      | Q2088324
| 1633      | Colonial Brazil    | 1815       |1816      | Q3932042
| 1634      | Colonial Brazil    | 1816       |1822      | Q3932042
| 1635      | Empire of Brazil   | 1822       |1889      | Q217230
| 1636      | Republic of the United States of Brazil   | 1889       |1967      | Q155
| 1637      | Federative Republic of Brazil  | 1967       |9999      | Q155





| capital_id | name               | start_date | end_date | wikidata
|-----------|--------------------|------------|----------|----------|
| 1      | Salvador   | 1540       |1763      | Q36947
| 2      | Rio de Janeiro   | 1763       |1960      | Q8678
| 3      | Brasília   | 1960       |9999      | Q2844
| 4      | Valletta   | 1974       |9999      | Q23800

## Region x Capital (Many-to-Many)

| region_id  | capital_id
|------------|-----------|
| 1628       | 4
| 1629       | 1
| 1630       | 1
| 1631       | 1
| 1632       | 1
| 1632       | 2
| 1633       | 2
| 1634       | 2
| 1635       | 2
| 1636       | 2
| 1636       | 3
| 1637       | 3

## The best way to represent these information in a CSV file?

**1. Representing in a single column, comma-separated string**

1,Salvador,1540,1763,Q36947,**1629,1630,1631,1632**

2,Rio de Janeiro,1763,1960,Q8678,**1632,1633,1634,1635,1636**

3,Brasilia,1960,9999,Q2844,**1636,1637**

4, Valletta,1974,9999,Q23800,**1628**

---
<font color='blue'>**2. Representing in a single column, as a list**</font>

<font color='blue'>1,Salvador,1540,1763,Q36947,**"1629,1630,1631,1632"**</font>

<font color='blue'>2,Rio de Janeiro,1763,1960,Q8678,**"1632,1633,1634,1635,1636"**</font>

<font color='blue'>3,Brasilia,1960,9999,Q2844,**"1636,1637"**</font>

<font color='blue'>4,Valletta,1974,9999,Q23800,**1628**</font>


---
**3. Representing in a single row the relation between Region and Capital**

1,Salvador,1540,1763,Q36947,**1629**

1,Salvador,1540,1763,Q36947,**1630**

1,Salvador,1540,1763,Q36947,**1631**

1,Salvador,1540,1763,Q36947,**1632**

2,Rio de Janeiro,1763,1960,Q8678,**1632**

2,Rio de Janeiro,1763,1960,Q8678,**1633**

2,Rio de Janeiro,1763,1960,Q8678,**1634**

2,Rio de Janeiro,1763,1960,Q8678,**1635**

2,Rio de Janeiro,1763,1960,Q8678,**1636**

3,Brasilia,1960,9999,Q2844,**1636**

3,Brasilia,1960,9999,Q2844,**1637**

4,Valletta,1974,9999,Q23800,**1628**



## Importante notes

1.   Some regions do **not** have a registered capital, and this information will not be included in the Capital CSV file.

    E.g.: 1619 - Monastic State of the Order of Malta
2.   Some capitals **do** not have a start_date or an end_date. Could I assume these dates the same than the region dates?

    E.g: 1928 - Republic of Malta


**Properties:**

*   description
*   coordinate location (https://epsg.io/)
*   languages link and text (en, cs, zh, jp, de, fr, it, es, pt, pl, ar, nl)
*   inception

# Importing libs and global configs

In [ ]:
from google.colab import files, drive
import pandas as pd
import json
import re
import requests
from typing import List, Dict, Union

In [ ]:
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
  'Accept': 'application/json'
}

sparql_endpoint = "https://query.wikidata.org/sparql"

In [ ]:
def save_csv(dataframe, file_name):
    file_path = '/content/gdrive/My Drive/'
    file_info = file_path + file_name
    drive.mount('/content/gdrive', force_remount=True)
    dataframe.to_csv(file_info, index=False)

In [ ]:
def grouping_column_values(column):

    # Selecting distinct column values entries
    distinct_wikidata = column.drop_duplicates().tolist()

    # Considerer only valid Wikidatas
    distinct_wikidata_clean = [valor for valor in distinct_wikidata if isinstance(valor, str) and valor.startswith('Q') and valor[1:].isdigit()]

    print(f'Total values: {len(distinct_wikidata_clean)}')

    # Grouping them into sets of up to 500 due to request size limitation
    MAX_SIZE = 500
    wikidata_groups = [distinct_wikidata_clean[i:i + MAX_SIZE] for i in range(0, len(distinct_wikidata_clean), MAX_SIZE)]

    # Checking each group total
    for index, group in enumerate(wikidata_groups):
        print(f'Group: {index} Quantity: {len(group)}')
    return wikidata_groups

In [ ]:
def filter_na(values_list):
    if isinstance(values_list, list):
        filtered_coords = [
            coord
            for coord in values_list
            if not any(pd.isna(x) for x in coord)
        ]
        return filtered_coords
    return values_list

In [ ]:
# Function to extract QID from the URI

def extract_q_number(url):
    match = re.search(r'/Q(\d+)', url)

    return f'Q{match.group(1)}' if match else None

In [ ]:
def extract_year(date_str):
    if pd.notna(date_str):
        try:
            match = re.match(r'(-?\d+)-\d{2}-\d{2}', date_str)
            if match:
                return int(match.group(1))
        except (IndexError, ValueError):
                return None
    return None

In [ ]:
def convert_to_int(valor):
    try:
        return int(valor)
    except (ValueError, TypeError):
        return None

In [ ]:
def is_valid_datetime(value) -> bool:
    if pd.notna(value):
        return True
    if extract_year(value):
        return True
    return False

In [ ]:
def extract_numbers(s):
    return set(int(num) for num in s.replace("{", "").replace("}", "").replace("[", "").replace("]", "").split(",") if num.strip().isdigit())

# Uploading regions file and checking data cleaning

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

Saving rulers.xlsx to rulers.xlsx


In [ ]:
# Selecting Regions sheet

data = pd.read_excel(file_name, sheet_name=None)
sheet = 'Regions'
regions_original_df = data[sheet]

In [ ]:
# Filtering only the needed columns from Region

fields_selected = ['region_id', 'name', 'start_date', 'end_date', 'wikidata']
regions_df = regions_original_df[fields_selected]

In [ ]:
regions_df

,region_id,name,start_date,end_date,wikidata
0,1.0,Yellow River civilization,-7500,-1400,Q28584
1,2.0,China,-1500,-1046,Q630276
2,3.0,Egypt,-6000,-3100,Q11768
3,4.0,Lower Egypt,-3500,-3100,Q11768
4,5.0,Egypt,-3100,-2686,Q11768
...,...,...,...,...,...
3247,4216.0,West Virginia,1863,1863,Q1371
3248,4217.0,West Virginia,1863,1863,Q1371
3249,4218.0,West Virginia,1863,9999,Q1371
3250,4219.0,Wyoming,1890,9999,Q1214


In [ ]:
# Dropping REGION_ID and WIKIDATA missing data

regions_df = regions_df.dropna(subset=['region_id', 'wikidata'])
regions_df

,region_id,name,start_date,end_date,wikidata
0,1.0,Yellow River civilization,-7500,-1400,Q28584
1,2.0,China,-1500,-1046,Q630276
2,3.0,Egypt,-6000,-3100,Q11768
3,4.0,Lower Egypt,-3500,-3100,Q11768
4,5.0,Egypt,-3100,-2686,Q11768
...,...,...,...,...,...
3247,4216.0,West Virginia,1863,1863,Q1371
3248,4217.0,West Virginia,1863,1863,Q1371
3249,4218.0,West Virginia,1863,9999,Q1371
3250,4219.0,Wyoming,1890,9999,Q1214


In [ ]:
# Converting region_id to integer

regions_df = regions_df.copy()
regions_df['region_id'] = pd.to_numeric(regions_df['region_id'], errors='coerce', downcast='integer')

In [ ]:
regions_df

,region_id,name,start_date,end_date,wikidata
0,1,Yellow River civilization,-7500,-1400,Q28584
1,2,China,-1500,-1046,Q630276
2,3,Egypt,-6000,-3100,Q11768
3,4,Lower Egypt,-3500,-3100,Q11768
4,5,Egypt,-3100,-2686,Q11768
...,...,...,...,...,...
3247,4216,West Virginia,1863,1863,Q1371
3248,4217,West Virginia,1863,1863,Q1371
3249,4218,West Virginia,1863,9999,Q1371
3250,4219,Wyoming,1890,9999,Q1214


# Fetching capital info from wikidata

In [ ]:
wikidata_regions_groups = grouping_column_values(regions_df['wikidata'])

Total values: 1710
Group: 0 Quantity: 500
Group: 1 Quantity: 500
Group: 2 Quantity: 500
Group: 3 Quantity: 210


In [ ]:
# Query to retrieve the capital(s) by region QID

def find_capital_by_qid(qids: list) -> Union[Dict, List]:
    values = " ".join(["wd:" + qid for qid in qids])

    sparql_query = f'''
        SELECT ?item ?capital ?capitalLabel ?startTime ?endTime
        WHERE {{
            VALUES ?item {{ {values} }}
            ?item wdt:P36 ?capital.
        OPTIONAL {{
            ?item p:P36 ?capitalStatement.
            ?capitalStatement ps:P36 ?capital.
            ?capitalStatement pq:P580 ?startTime.
            OPTIONAL {{ ?capitalStatement pq:P582 ?endTime. }}
        }}
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
    '''

    params = {
      'query': sparql_query,
      'format': 'json'
    }

    try:
        response = requests.get(sparql_endpoint, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            bindings = data['results']['bindings']
            return bindings

        return {"error": { "code": response.status_code, "message": response.text}}

    except requests.exceptions.RequestException as e:
        return {"error": e}

In [ ]:
# Creating a list that contains the result from the query above

bindings_list = []

for index, group in enumerate(wikidata_regions_groups):
  print(f'Group: {index} QID Quantity: {len(group)}')
  bindings = find_capital_by_qid(group)
  print(f'Results found: {len(bindings)}')
  print(f'Results: {bindings}')
  print('-----------------------------------')
  bindings_list.extend(bindings)

Group: 0 QID Quantity: 500
Results found: 424
Results: [{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q41642'}, 'capital': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q82070'}, 'capitalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Tyre'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q41137'}, 'capital': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q165841'}, 'capitalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Kar-Tukulti-Ninurta'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q41642'}, 'capital': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q163490'}, 'capitalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Sidon'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q41137'}, 'capital': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q199547'}, 'capitalLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Harran'}}, {'item': {'type

In [ ]:
# Converting the query response into a data frame

capitals_df = pd.json_normalize(bindings_list)
capitals_df

,item.type,item.value,capital.type,capital.value,capitalLabel.xml:lang,capitalLabel.type,capitalLabel.value,startTime.datatype,startTime.type,startTime.value,endTime.datatype,endTime.type,endTime.value
0,uri,http://www.wikidata.org/entity/Q41642,uri,http://www.wikidata.org/entity/Q82070,en,literal,Tyre,NaN,NaN,NaN,NaN,NaN,NaN
1,uri,http://www.wikidata.org/entity/Q41137,uri,http://www.wikidata.org/entity/Q165841,en,literal,Kar-Tukulti-Ninurta,NaN,NaN,NaN,NaN,NaN,NaN
2,uri,http://www.wikidata.org/entity/Q41642,uri,http://www.wikidata.org/entity/Q163490,en,literal,Sidon,NaN,NaN,NaN,NaN,NaN,NaN
3,uri,http://www.wikidata.org/entity/Q41137,uri,http://www.wikidata.org/entity/Q199547,en,literal,Harran,NaN,NaN,NaN,NaN,NaN,NaN
4,uri,http://www.wikidata.org/entity/Q41137,uri,http://www.wikidata.org/entity/Q200200,en,literal,Assur,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,uri,http://www.wikidata.org/entity/Q1117425,uri,http://www.wikidata.org/entity/Q37066,en,literal,Bismarck,http://www.w3.org/2001/XMLSchema#dateTime,literal,1883-01-01T00:00:00Z,http://www.w3.org/2001/XMLSchema#dateTime,literal,1889-11-02T00:00:00Z
1628,uri,http://www.wikidata.org/entity/Q865,uri,http://www.wikidata.org/entity/Q1867,en,literal,Taipei,http://www.w3.org/2001/XMLSchema#dateTime,literal,1949-12-07T00:00:00Z,NaN,NaN,NaN
1629,uri,http://www.wikidata.org/entity/Q1008,uri,http://www.wikidata.org/entity/Q3768,en,literal,Yamoussoukro,http://www.w3.org/2001/XMLSchema#dateTime,literal,1983-03-01T00:00:00Z,NaN,NaN,NaN
1630,uri,http://www.wikidata.org/entity/Q1020,uri,http://www.wikidata.org/entity/Q3876,en,literal,Lilongwe,http://www.w3.org/2001/XMLSchema#dateTime,literal,1975-01-01T00:00:00Z,NaN,NaN,NaN


In [ ]:
# Selecting only the columns needed

capitals_df = capitals_df.loc[:, ['item.value', 'capital.value', 'capitalLabel.value', 'startTime.value', 'endTime.value']]
capitals_df = capitals_df.rename(columns={'item.value': 'region_uri', 'capital.value': 'capital_uri','capitalLabel.value': 'capital_name', 'startTime.value': 'start_time', 'endTime.value':'end_time'})

In [ ]:
# Extracting WIKIDATA code from URI

capitals_df['wikidata_capital'] = capitals_df['capital_uri'].apply(extract_q_number)
capitals_df['wikidata_region'] = capitals_df['region_uri'].apply(extract_q_number)

In [ ]:
# Removing unnecessary columns

capitals_df = capitals_df.drop(columns=['region_uri', 'capital_uri'])

In [ ]:
# Reordering columns

new_columns_order = ['wikidata_capital', 'capital_name', 'start_time', 'end_time', 'wikidata_region']
capitals_df = capitals_df[new_columns_order]

In [ ]:
capitals_df

,wikidata_capital,capital_name,start_time,end_time,wikidata_region
0,Q82070,Tyre,NaN,NaN,Q41642
1,Q165841,Kar-Tukulti-Ninurta,NaN,NaN,Q41137
2,Q163490,Sidon,NaN,NaN,Q41642
3,Q199547,Harran,NaN,NaN,Q41137
4,Q200200,Assur,NaN,NaN,Q41137
...,...,...,...,...,...
1627,Q37066,Bismarck,1883-01-01T00:00:00Z,1889-11-02T00:00:00Z,Q1117425
1628,Q1867,Taipei,1949-12-07T00:00:00Z,NaN,Q865
1629,Q3768,Yamoussoukro,1983-03-01T00:00:00Z,NaN,Q1008
1630,Q3876,Lilongwe,1975-01-01T00:00:00Z,NaN,Q1020


# Adding **region_id** to Capital


In [ ]:
# Iterate over the CAPITAL dataframe to find corresponding regions_id
# using REGIONS WIKIDATA as a link

def add_region_id_to_capital(subset_regions_df):
    region_id_list = subset_regions_df['region_id'].to_list()
    capitals_df.loc[index, 'region_id'] = str(region_id_list)

for index, row in capitals_df.iterrows():
    wikidata_region = row['wikidata_region']
    capital_start_time = row['start_time']
    capital_end_time = row['end_time']
    subset_regions_df = regions_df.query('wikidata == @wikidata_region')
    is_capital_linked_to_a_single_region_id = subset_regions_df.shape[0] == 1

    if is_capital_linked_to_a_single_region_id:
        add_region_id_to_capital(subset_regions_df)
    elif is_valid_datetime(capital_start_time) and is_valid_datetime(capital_end_time):
        # Some regions might have the same wikidata
        # And the capital is related to the wikidata_region.
        # If there is a capital start and end time, it is important to considerer also the region start and end date before linking to region_id

        subset_regions_df = subset_regions_df.copy()
        subset_regions_df['start_date'] = subset_regions_df['start_date'].astype(int)
        subset_regions_df['end_date'] = subset_regions_df['end_date'].astype(int)
        capital_start = extract_year(capital_start_time)
        capital_end = extract_year(capital_end_time)

        regions_date_intersection_df = subset_regions_df[(subset_regions_df['start_date'] <= capital_end) & (subset_regions_df['end_date'] >= capital_start)]
        add_region_id_to_capital(regions_date_intersection_df)
    else:
        add_region_id_to_capital(subset_regions_df)

In [ ]:
capitals_df = capitals_df.sort_values(by=['capital_name', 'start_time', 'end_time'])
capitals_df

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
364,Q1017,Aachen,0768-01-01T00:00:00Z,0843-01-01T00:00:00Z,Q146246,[528]
324,Q1017,Aachen,0962-01-01T00:00:00Z,1346-01-01T00:00:00Z,Q12548,"[827, 828, 829]"
68,Q1017,Aachen,NaN,NaN,Q468902,[833]
471,Q14274,Aarau,NaN,NaN,Q206696,[1693]
1294,Q189685,Abomey,NaN,NaN,Q468814,"[2413, 2414]"
...,...,...,...,...,...,...
52,Q6635525,list of provinces of Balhae,NaN,NaN,Q28322,"[662, 663, 664]"
754,Q16656266,İske Qazan museum-preserve,NaN,NaN,Q186537,[1461]
1005,Q580,Łódź,NaN,NaN,Q156111,[2631]
84,Q3549,Šibenik,NaN,NaN,Q858841,"[726, 727]"


Grouping capitals with same start_time and end_time but linked to different wikidata_region

In [ ]:
# Example

value = 'Q8678'
capitals_df.query('wikidata_capital == @value')

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
928,Q8678,Rio de Janeiro,1763-01-01T00:00:00Z,1815-01-01T00:00:00Z,Q2088324,[1632]
393,Q8678,Rio de Janeiro,1808-01-01T00:00:00Z,1822-01-01T00:00:00Z,Q45670,"[1114, 1115]"
480,Q8678,Rio de Janeiro,NaN,NaN,Q217230,[1635]
577,Q8678,Rio de Janeiro,NaN,NaN,Q3932042,"[1633, 1634]"


In [ ]:
# Preparing dataframe before grouping

copied_capitals_df = capitals_df.copy()
copied_capitals_df['wikidata_capital'].fillna('None', inplace=True)
copied_capitals_df.loc[copied_capitals_df['wikidata_capital'] == 'None', 'capital_name'] = 'None'
copied_capitals_df['start_time'].fillna('None', inplace=True)
copied_capitals_df['end_time'].fillna('None', inplace=True)
copied_capitals_df['region_id'] = copied_capitals_df['region_id'].apply(extract_numbers)

In [ ]:
copied_capitals_df.head()

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
364,Q1017,Aachen,0768-01-01T00:00:00Z,0843-01-01T00:00:00Z,Q146246,{528}
324,Q1017,Aachen,0962-01-01T00:00:00Z,1346-01-01T00:00:00Z,Q12548,"{827, 828, 829}"
68,Q1017,Aachen,None,None,Q468902,{833}
471,Q14274,Aarau,None,None,Q206696,{1693}
1294,Q189685,Abomey,None,None,Q468814,"{2413, 2414}"


In [ ]:
capitals_df_grouped = copied_capitals_df.groupby(['wikidata_capital', 'capital_name', 'start_time', 'end_time'])['region_id'].agg(lambda x: list(set.union(*x))).reset_index()

In [ ]:
# Example after grouping

value = 'Q8678'
capitals_df_grouped.query('wikidata_capital == @value')

,wikidata_capital,capital_name,start_time,end_time,region_id
1109,Q8678,Rio de Janeiro,1763-01-01T00:00:00Z,1815-01-01T00:00:00Z,[1632]
1110,Q8678,Rio de Janeiro,1808-01-01T00:00:00Z,1822-01-01T00:00:00Z,"[1114, 1115]"
1111,Q8678,Rio de Janeiro,None,None,"[1633, 1634, 1635]"


In [ ]:
capitals_df_grouped

,wikidata_capital,capital_name,start_time,end_time,region_id
0,None,None,None,None,[2011]
1,Q100,Boston,None,None,"[4080, 4076, 4077, 4078, 4079]"
2,Q1000918,Kaiyuan,None,None,[936]
3,Q10054424,Fengtian Fu,1625-01-01T00:00:00Z,1644-01-01T00:00:00Z,[1737]
4,Q10054424,Fengtian Fu,1644-01-01T00:00:00Z,1905-01-01T00:00:00Z,"[1737, 1738, 1739, 1740]"
...,...,...,...,...,...
1153,Q987,New Delhi,1911-01-01T00:00:00Z,1947-01-01T00:00:00Z,[2053]
1154,Q987,New Delhi,None,None,"[2054, 2055]"
1155,Q989288,Yankton,1861-01-01T00:00:00Z,1883-01-01T00:00:00Z,"[4032, 4033, 4034, 4035, 4036, 4029, 4030, 4031]"
1156,Q994,Tbilisi,1122-01-01T00:00:00Z,None,"[2724, 2725, 2726, 2727]"


# Fetching capital **inception dates** and **coordinates**

In [ ]:
wikidata_capitals_groups = grouping_column_values(capitals_df_grouped['wikidata_capital'])

Total values: 960
Group: 0 Quantity: 500
Group: 1 Quantity: 460


In [ ]:
# Query to retrieve INCEPTION DATES and COORDINATES by capital QID

def find_capital_inception_coordinates(qids: list) -> Union[Dict, List]:
    values = " ".join(["wd:" + qid for qid in qids])

    sparql_query = f'''
        SELECT ?item ?itemLabel ?itemDescription ?inceptionDate ?coordinates
               ?altLabel
        WHERE {{
            VALUES ?item {{ {values} }}
            OPTIONAL {{ ?item wdt:P571 ?inceptionDate. }}
            OPTIONAL {{ ?item wdt:P625 ?coordinates. }}
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
    '''

    params = {
        'query': sparql_query,
        'format': 'json'
    }

    try:
        response = requests.get(sparql_endpoint, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            bindings = data['results']['bindings']
            return bindings

        return {"error": { "code": response.status_code, "message": response.text}}

    except requests.exceptions.RequestException as e:
        return {"error": e}

In [ ]:
# Creating a list that contains the result from the query above

bindings_list = []

for index, group in enumerate(wikidata_capitals_groups):
  print(f'Group: {index} QID Quantity: {len(group)}')
  bindings = find_capital_inception_coordinates(group)
  print(f'Results found: {len(bindings)}')
  print(f'Results: {bindings}')
  print('-----------------------------------')
  bindings_list.extend(bindings)

Group: 0 QID Quantity: 500
Results found: 511
Results: [{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q100'}, 'inceptionDate': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1630-09-17T00:00:00Z'}, 'coordinates': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-71.057777777 42.360277777)'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Boston'}, 'itemDescription': {'xml:lang': 'en', 'type': 'literal', 'value': 'capital and largest city of Massachusetts, United States'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q172'}, 'inceptionDate': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1750-01-01T00:00:00Z'}, 'coordinates': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-79.386666666 43.670277777)'}, 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 

In [ ]:
# Converting the query response into a data frame

capitals_inception_coordinate_df = pd.json_normalize(bindings_list)
capitals_inception_coordinate_df

,item.type,item.value,inceptionDate.datatype,inceptionDate.type,inceptionDate.value,coordinates.datatype,coordinates.type,coordinates.value,itemLabel.xml:lang,itemLabel.type,itemLabel.value,itemDescription.xml:lang,itemDescription.type,itemDescription.value
0,uri,http://www.wikidata.org/entity/Q100,http://www.w3.org/2001/XMLSchema#dateTime,literal,1630-09-17T00:00:00Z,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(-71.057777777 42.360277777),en,literal,Boston,en,literal,"capital and largest city of Massachusetts, Uni..."
1,uri,http://www.wikidata.org/entity/Q172,http://www.w3.org/2001/XMLSchema#dateTime,literal,1750-01-01T00:00:00Z,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(-79.386666666 43.670277777),en,literal,Toronto,en,literal,capital and largest city of the province of On...
2,uri,http://www.wikidata.org/entity/Q174,http://www.w3.org/2001/XMLSchema#dateTime,literal,1554-02-04T00:00:00Z,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(-46.633947222 -23.550394444),en,literal,São Paulo,en,literal,most populous city in Brazil
3,uri,http://www.wikidata.org/entity/Q216,http://www.w3.org/2001/XMLSchema#dateTime,literal,1201-01-01T00:00:00Z,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(25.28 54.687222222),en,literal,Vilnius,en,literal,capital and largest city of Lithuania
4,uri,http://www.wikidata.org/entity/Q220,http://www.w3.org/2001/XMLSchema#dateTime,literal,-0752-04-13T00:00:00Z,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(12.482777777 41.893055555),en,literal,Rome,en,literal,capital and largest city of Italy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,uri,http://www.wikidata.org/entity/Q46451169,NaN,NaN,NaN,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(106.55 29.55),en,literal,Chungking,en,literal,former English name of Chongqing
979,uri,http://www.wikidata.org/entity/Q46452338,http://www.w3.org/2001/XMLSchema#dateTime,literal,1921-01-01T00:00:00Z,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(113.258976111 23.128795),en,literal,Canton,en,literal,Guangzhou during the Republic of China (1912-1...
980,uri,http://www.wikidata.org/entity/Q60845861,NaN,NaN,NaN,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(1.443888888 43.604438888),en,literal,Tolosa,en,literal,"ancient Roman city in Toulouse, France"
981,uri,http://www.wikidata.org/entity/Q63256193,NaN,NaN,NaN,http://www.opengis.net/ont/geosparql#wktLiteral,literal,Point(-4.022630555 39.8581),en,literal,Tulaytula,en,literal,city in Al-Andalus; modern Toledo


In [ ]:
# Selecting only the columns needed

capitals_inception_coordinate_df = capitals_inception_coordinate_df.loc[:, ['item.value', 'inceptionDate.value', 'coordinates.value', 'itemDescription.value']]
capitals_inception_coordinate_df = capitals_inception_coordinate_df.rename(
    columns={'item.value': 'capital_uri', 'inceptionDate.value': 'inception_date','coordinates.value': 'coordinates', 'itemDescription.value': 'description'}
)

In [ ]:
capitals_inception_coordinate_df['wikidata_capital'] = capitals_inception_coordinate_df['capital_uri'].apply(extract_q_number)

In [ ]:
# Removing unnecessary columns

capitals_inception_coordinate_df = capitals_inception_coordinate_df.drop(columns=['capital_uri'])

In [ ]:
# Reordering columns

capitals_inception_coordinate_df = capitals_inception_coordinate_df.sort_values(by=['wikidata_capital'])

In [ ]:
capitals_inception_coordinate_df

,inception_date,coordinates,description,wikidata_capital
0,1630-09-17T00:00:00Z,Point(-71.057777777 42.360277777),"capital and largest city of Massachusetts, Uni...",Q100
366,NaN,Point(124.03713 42.53795),county-level city in Tieling,Q1000918
473,1644-01-01T00:00:00Z,Point(123.41988 41.78825),old capital of Qing Dynasty (1625-1644) before...,Q10054424
367,NaN,Point(88.365736111 23.408822222),"Heritage City in West Bengal, India",Q1006330
106,NaN,Point(76.653055555 12.308611111),"city in the state of Karnataka, India",Q10086
...,...,...,...,...
932,NaN,Point(106.07931 33.76944),"county in Longnan, Gansu, China",Q974580
933,1788-01-01T00:00:00Z,Point(-81.45028 39.42056),"city in and county seat of Washington County, ...",Q985482
540,1911-01-01T00:00:00Z,Point(77.2090057 28.6138954),capital city of India,Q987
934,1858-01-01T00:00:00Z,Point(-97.396694444 42.871305555),"county seat of Yankton County, South Dakota, U...",Q989288


In [ ]:
# Create dataframe with COORDINATES info

capitals_coordinate_df = capitals_inception_coordinate_df.drop(columns=['inception_date', 'description'], inplace=False)
capitals_coordinate_df = capitals_coordinate_df.drop_duplicates()
capitals_coordinate_df = capitals_coordinate_df.dropna(subset=['coordinates'])

In [ ]:
capitals_grouped_by_coordinate_df = capitals_coordinate_df.groupby(['wikidata_capital'])['coordinates'].agg(list).reset_index()
capitals_grouped_by_coordinate_df

,wikidata_capital,coordinates
0,Q100,[Point(-71.057777777 42.360277777)]
1,Q1000918,[Point(124.03713 42.53795)]
2,Q10054424,[Point(123.41988 41.78825)]
3,Q1006330,[Point(88.365736111 23.408822222)]
4,Q10086,[Point(76.653055555 12.308611111)]
...,...,...
928,Q974580,[Point(106.07931 33.76944)]
929,Q985482,[Point(-81.45028 39.42056)]
930,Q987,[Point(77.2090057 28.6138954)]
931,Q989288,[Point(-97.396694444 42.871305555)]


In [ ]:
# Create dataframe with INCEPTION DATES info

capitals_inception_df = capitals_inception_coordinate_df.drop(columns=['coordinates', 'description'], inplace=False)
capitals_inception_df = capitals_inception_df.drop_duplicates()
capitals_inception_df = capitals_inception_df.dropna(subset=['inception_date'])

In [337]:
capitals_grouped_by_inception_df = capitals_inception_df.groupby(['wikidata_capital'])['inception_date'].agg(list).reset_index()
capitals_grouped_by_inception_df

,wikidata_capital,inception_date
0,Q100,[1630-09-17T00:00:00Z]
1,Q10054424,[1644-01-01T00:00:00Z]
2,Q1013421,[1873-01-01T00:00:00Z]
3,Q1014248,[1553-01-01T00:00:00Z]
4,Q101625,[1065-01-01T00:00:00Z]
...,...,...
442,Q973656,[1471-01-01T00:00:00Z]
443,Q985482,[1788-01-01T00:00:00Z]
444,Q987,[1911-01-01T00:00:00Z]
445,Q989288,[1858-01-01T00:00:00Z]


# Fetching languages link and text

In [352]:
# Query to retrieve LANGUAGES LINK by capital QID

def find_capital_link_languages(qids: list) -> Union[Dict, List]:
    values = " ".join(["wd:" + qid for qid in qids])

    sparql_query = f'''
        SELECT ?item ?article
        WHERE {{
            VALUES ?item {{ {values} }}
            OPTIONAL {{
                ?item wdt:P123 ?article.
                ?article schema:about ?wikiLink.
             }}
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
    '''

    params = {
        'query': sparql_query,
        'format': 'json'
    }

    try:
        response = requests.get(sparql_endpoint, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            print(data)
            bindings = data['results']['bindings']
            return bindings

        return {"error": { "code": response.status_code, "message": response.text}}

    except requests.exceptions.RequestException as e:
        return {"error": e}

In [353]:
# Creating a list that contains the result from the query above

bindings_list = []

for index, group in enumerate(wikidata_capitals_groups):
  print(f'Group: {index} QID Quantity: {len(group)}')
  bindings = find_capital_link_languages(group)
  print(f'Results found: {len(bindings)}')
  print(f'Results: {bindings}')
  print('-----------------------------------')
  bindings_list.extend(bindings)

Group: 0 QID Quantity: 500
{'head': {'vars': ['item', 'article']}, 'results': {'bindings': [{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q101418'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q101625'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q102158'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q105284'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q107370'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q107401'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q108155'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q109079'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q111705'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q119140'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q121157'}}, {'item': {'type': 'uri', 'value': 'http://www.wiki

In [355]:
wikidata_capitals_groups

[['Q100',
  'Q1000918',
  'Q10054424',
  'Q1006330',
  'Q10086',
  'Q101186473',
  'Q1012971',
  'Q1012975',
  'Q1013421',
  'Q1013556',
  'Q101418',
  'Q1014248',
  'Q101625',
  'Q1017',
  'Q1017178',
  'Q1021154',
  'Q102158',
  'Q1022',
  'Q1025459',
  'Q1025475',
  'Q1031919',
  'Q1040',
  'Q10400',
  'Q10439273',
  'Q105076255',
  'Q105284',
  'Q1061338',
  'Q10690',
  'Q1070000',
  'Q10704',
  'Q10717',
  'Q107370',
  'Q107401',
  'Q1074347',
  'Q107585355',
  'Q108155',
  'Q10825974',
  'Q1083',
  'Q1085',
  'Q10885263',
  'Q10906766',
  'Q109079',
  'Q10923028',
  'Q1093156',
  'Q11065738',
  'Q11122735',
  'Q111705',
  'Q111905239',
  'Q11194',
  'Q1131299',
  'Q1136681',
  'Q11443',
  'Q11445',
  'Q1149666',
  'Q1152177',
  'Q115738657',
  'Q116158888',
  'Q116224697',
  'Q11720',
  'Q11739',
  'Q1184661',
  'Q1189121',
  'Q118941048',
  'Q1190403',
  'Q11907651',
  'Q119140',
  'Q11930644',
  'Q1193412',
  'Q1194737',
  'Q11947752',
  'Q1196479',
  'Q11974',
  'Q1198957',
  

In [354]:
# Converting the query response into a data frame

teste = pd.json_normalize(bindings_list)
teste

,item.type,item.value
0,uri,http://www.wikidata.org/entity/Q101418
1,uri,http://www.wikidata.org/entity/Q101625
2,uri,http://www.wikidata.org/entity/Q102158
3,uri,http://www.wikidata.org/entity/Q105284
4,uri,http://www.wikidata.org/entity/Q107370
...,...,...
955,uri,http://www.wikidata.org/entity/Q7856199
956,uri,http://www.wikidata.org/entity/Q7929221
957,uri,http://www.wikidata.org/entity/Q7929348
958,uri,http://www.wikidata.org/entity/Q8075286


# Exporting csv

In [ ]:
other_props_df = pd.merge(grouped_df_coordinates, grouped_df_inception, on=['wikidata_capital', 'description'], how='inner')
other_props_df

,wikidata_capital,description,coordinates,inception_date
0,Q100,"capital and largest city of Massachusetts, Uni...",Point(-71.057777777 42.360277777),1630-09-17T00:00:00Z
1,Q1000918,county-level city in Tieling,Point(124.03713 42.53795),NaN
2,Q10054424,old capital of Qing Dynasty (1625-1644) before...,Point(123.41988 41.78825),1644-01-01T00:00:00Z
3,Q1006330,"Heritage City in West Bengal, India",Point(88.365736111 23.408822222),NaN
4,Q10086,"city in the state of Karnataka, India",Point(76.653055555 12.308611111),NaN
...,...,...,...,...
943,Q974580,"county in Longnan, Gansu, China",Point(106.07931 33.76944),NaN
944,Q985482,"city in and county seat of Washington County, ...",Point(-81.45028 39.42056),1788-01-01T00:00:00Z
945,Q987,capital city of India,Point(77.2090057 28.6138954),1911-01-01T00:00:00Z
946,Q989288,"county seat of Yankton County, South Dakota, U...",Point(-97.396694444 42.871305555),1858-01-01T00:00:00Z


In [ ]:
grouped_df_inception = new_capitals_df.groupby(['wikidata_capital', 'description'])['inception_date'].agg(aggregate_coordinates).reset_index()
grouped_df_inception

,wikidata_capital,description,inception_date
0,Q100,"capital and largest city of Massachusetts, Uni...",1630-09-17T00:00:00Z
1,Q1000918,county-level city in Tieling,NaN
2,Q10054424,old capital of Qing Dynasty (1625-1644) before...,1644-01-01T00:00:00Z
3,Q1006330,"Heritage City in West Bengal, India",NaN
4,Q10086,"city in the state of Karnataka, India",NaN
...,...,...,...
943,Q974580,"county in Longnan, Gansu, China",NaN
944,Q985482,"city in and county seat of Washington County, ...",1788-01-01T00:00:00Z
945,Q987,capital city of India,1911-01-01T00:00:00Z
946,Q989288,"county seat of Yankton County, South Dakota, U...",1858-01-01T00:00:00Z


In [ ]:
value = 'Q454949'
other_props_df.query('wikidata_capital == @value')

,wikidata_capital,description,coordinates,inception_date
681,Q454949,"town in Mandalay, Myanmar","[Point(96.045833333 21.9), Point(96.09545 21.8...","[1783-01-01T00:00:00Z, 1783-01-01T00:00:00Z]"


In [ ]:
duplicated_rows = other_props_df.duplicated(subset=['wikidata_capital'], keep=False)
duplicated_capitals_df = other_props_df[duplicated_rows]
duplicated_capitals_df

,wikidata_capital,description,coordinates,inception_date


In [ ]:
other_props_df

,wikidata_capital,description,coordinates,inception_date
0,Q100,"capital and largest city of Massachusetts, Uni...",[Point(-71.057777777 42.360277777)],[1630-09-17T00:00:00Z]
1,Q1000918,county-level city in Tieling,[Point(124.03713 42.53795)],[nan]
2,Q10054424,old capital of Qing Dynasty (1625-1644) before...,[Point(123.41988 41.78825)],[1644-01-01T00:00:00Z]
3,Q1006330,"Heritage City in West Bengal, India",[Point(88.365736111 23.408822222)],[nan]
4,Q10086,"city in the state of Karnataka, India",[Point(76.653055555 12.308611111)],[nan]
...,...,...,...,...
943,Q974580,"county in Longnan, Gansu, China",[Point(106.07931 33.76944)],[nan]
944,Q985482,"city in and county seat of Washington County, ...",[Point(-81.45028 39.42056)],[1788-01-01T00:00:00Z]
945,Q987,capital city of India,[Point(77.2090057 28.6138954)],[1911-01-01T00:00:00Z]
946,Q989288,"county seat of Yankton County, South Dakota, U...",[Point(-97.396694444 42.871305555)],[1858-01-01T00:00:00Z]


In [ ]:
# Include coordinates and inception as a new column at capitals

merged_df = pd.merge(df_grouped, other_props_df, on='wikidata_capital', how='left')
merged_df

,wikidata_capital,capital_name,start_time,end_time,region_id,description,coordinates,inception_date
0,Q1017,Aachen,0768-01-01T00:00:00Z,0843-01-01T00:00:00Z,[528],"city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),NaN
1,Q1017,Aachen,0962-01-01T00:00:00Z,1346-01-01T00:00:00Z,"[827, 828, 829]","city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),NaN
2,Q1017,Aachen,NaN,NaN,[833],"city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),NaN
3,Q14274,Aarau,NaN,NaN,[1693],"capital of Canton Aargau, Switzerland",Point(8.0446 47.3923),1240-01-01T00:00:00Z
4,Q189685,Abomey,NaN,NaN,"[2413, 2414]",city in the Zou Department of Benin,Point(1.991388888 7.185277777),NaN
...,...,...,...,...,...,...,...,...
1152,Q6635525,list of provinces of Balhae,NaN,NaN,"[664, 662, 663]",Wikimedia list article,NaN,NaN
1153,Q16656266,İske Qazan museum-preserve,NaN,NaN,[1461],"historic, architectural and natural museum-pre...",Point(49.650535 56.025458),1992-01-01T00:00:00Z
1154,Q580,Łódź,NaN,NaN,[2631],city in Łódź Voivodeship in central Poland,Point(19.454722222 51.776944444),1423-08-07T00:00:00Z
1155,Q3549,Šibenik,NaN,NaN,"[726, 727]","city and settlement in Šibenik-Knin County, Cr...",Point(15.895555555 43.733888888),0860-01-01T00:00:00Z


In [ ]:
import numpy as np
df = merged_df.applymap(lambda x: '' if np.any(pd.isna(x) if isinstance(x, np.ndarray) else x == 'NaN') else x)

In [ ]:
df['inception_date'].fillna('', inplace=True)
df['coordinates'].fillna('', inplace=True)

In [ ]:
df

,wikidata_capital,capital_name,start_time,end_time,region_id,description,coordinates,inception_date
0,Q1017,Aachen,0768-01-01T00:00:00Z,0843-01-01T00:00:00Z,[528],"city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),
1,Q1017,Aachen,0962-01-01T00:00:00Z,1346-01-01T00:00:00Z,"[827, 828, 829]","city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),
2,Q1017,Aachen,,,[833],"city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),
3,Q14274,Aarau,,,[1693],"capital of Canton Aargau, Switzerland",Point(8.0446 47.3923),1240-01-01T00:00:00Z
4,Q189685,Abomey,,,"[2413, 2414]",city in the Zou Department of Benin,Point(1.991388888 7.185277777),
...,...,...,...,...,...,...,...,...
1152,Q6635525,list of provinces of Balhae,,,"[664, 662, 663]",Wikimedia list article,,
1153,Q16656266,İske Qazan museum-preserve,,,[1461],"historic, architectural and natural museum-pre...",Point(49.650535 56.025458),1992-01-01T00:00:00Z
1154,Q580,Łódź,,,[2631],city in Łódź Voivodeship in central Poland,Point(19.454722222 51.776944444),1423-08-07T00:00:00Z
1155,Q3549,Šibenik,,,"[726, 727]","city and settlement in Šibenik-Knin County, Cr...",Point(15.895555555 43.733888888),0860-01-01T00:00:00Z


In [ ]:
df['capital_id'] = df.index

In [ ]:
new_columns_order = ['capital_id', 'capital_name', 'wikidata_capital', 'description', 'coordinates', 'inception_date' , 'start_time', 'end_time', 'region_id']
ready_capitals_df = df[new_columns_order]

In [ ]:
ready_capitals_df

,capital_id,capital_name,wikidata_capital,description,coordinates,inception_date,start_time,end_time,region_id
0,0,Aachen,Q1017,"city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),,0768-01-01T00:00:00Z,0843-01-01T00:00:00Z,[528]
1,1,Aachen,Q1017,"city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),,0962-01-01T00:00:00Z,1346-01-01T00:00:00Z,"[827, 828, 829]"
2,2,Aachen,Q1017,"city in North Rhine-Westphalia, Germany",Point(6.083788 50.776207),,,,[833]
3,3,Aarau,Q14274,"capital of Canton Aargau, Switzerland",Point(8.0446 47.3923),1240-01-01T00:00:00Z,,,[1693]
4,4,Abomey,Q189685,city in the Zou Department of Benin,Point(1.991388888 7.185277777),,,,"[2413, 2414]"
...,...,...,...,...,...,...,...,...,...
1152,1152,list of provinces of Balhae,Q6635525,Wikimedia list article,,,,,"[664, 662, 663]"
1153,1153,İske Qazan museum-preserve,Q16656266,"historic, architectural and natural museum-pre...",Point(49.650535 56.025458),1992-01-01T00:00:00Z,,,[1461]
1154,1154,Łódź,Q580,city in Łódź Voivodeship in central Poland,Point(19.454722222 51.776944444),1423-08-07T00:00:00Z,,,[2631]
1155,1155,Šibenik,Q3549,"city and settlement in Šibenik-Knin County, Cr...",Point(15.895555555 43.733888888),0860-01-01T00:00:00Z,,,"[726, 727]"


In [ ]:
from google.colab import drive

# Monta o Google Drive
# drive.mount('/content/gdrive', force_remount=True)

# Salva o DataFrame no Google Drive
ready_capitals_df.to_csv('/content/gdrive/My Drive/capitals.csv', index=False)

In [ ]:
# VERIFY

value = 'Q467627'
capitals_df.query('wikidata_region == @value')

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
490,None,http://www.wikidata.org/.well-known/genid/5cb4...,NaN,NaN,Q467627,[2011]
859,Q5838,Kabul,1776-01-01T00:00:00Z,1823-01-01T00:00:00Z,Q467627,[2011]
860,Q5838,Kabul,1839-01-01T00:00:00Z,1842-01-01T00:00:00Z,Q467627,[2011]
861,Q45604,Kandahar,1747-01-01T00:00:00Z,1776-01-01T00:00:00Z,Q467627,[2011]


# Some regions do **not** have a Capital. How many are?

In [ ]:
capitals_df

,wikidata_capital,capital_name,start_time,end_time,wikidata_region
0,Q82070,Tyre,NaN,NaN,Q41642
1,Q165841,Kar-Tukulti-Ninurta,NaN,NaN,Q41137
2,Q163490,Sidon,NaN,NaN,Q41642
3,Q199547,Harran,NaN,NaN,Q41137
4,Q200200,Assur,NaN,NaN,Q41137
...,...,...,...,...,...
1627,Q35775,Boise,NaN,NaN,Q1221
1628,Q37066,Bismarck,NaN,NaN,Q1207
1629,Q28198,Jackson,NaN,NaN,Q1494
1630,Q28848,Saint Paul,NaN,NaN,Q1527


In [ ]:
regions_df

,region_id,name,start_date,end_date,wikidata,has_capital
0,1,Yellow River civilization,-7500,-1400,Q28584,False
1,2,China,-1500,-1046,Q630276,False
2,3,Egypt,-6000,-3100,Q11768,False
3,4,Lower Egypt,-3500,-3100,Q11768,False
4,5,Egypt,-3100,-2686,Q11768,False
...,...,...,...,...,...,...
3247,4216,West Virginia,1863,1863,Q1371,True
3248,4217,West Virginia,1863,1863,Q1371,True
3249,4218,West Virginia,1863,9999,Q1371,True
3250,4219,Wyoming,1890,9999,Q1214,True


In [ ]:
# Check if at least region has one capital registered

regions_df['has_capital'] = regions_df['wikidata'].isin(capitals_df['wikidata_region'])
regions_df

,region_id,name,start_date,end_date,wikidata,has_capital
0,1,Yellow River civilization,-7500,-1400,Q28584,False
1,2,China,-1500,-1046,Q630276,False
2,3,Egypt,-6000,-3100,Q11768,False
3,4,Lower Egypt,-3500,-3100,Q11768,False
4,5,Egypt,-3100,-2686,Q11768,False
...,...,...,...,...,...,...
3247,4216,West Virginia,1863,1863,Q1371,True
3248,4217,West Virginia,1863,1863,Q1371,True
3249,4218,West Virginia,1863,9999,Q1371,True
3250,4219,Wyoming,1890,9999,Q1214,True


In [ ]:
# Double checking if the data returned expected results

regions_df.query('has_capital == @False')

,region_id,name,start_date,end_date,wikidata,has_capital
0,1,Yellow River civilization,-7500,-1400,Q28584,False
1,2,China,-1500,-1046,Q630276,False
2,3,Egypt,-6000,-3100,Q11768,False
3,4,Lower Egypt,-3500,-3100,Q11768,False
4,5,Egypt,-3100,-2686,Q11768,False
...,...,...,...,...,...,...
3181,4150,Oregon Country,1818,1846,Q388164,False
3183,4152,Oregon Territory,1848,1853,Q1420228,False
3184,4153,Oregon Territory,1853,1859,Q1420228,False
3185,4154,Oregon Unorganized Federal Territory,1846,1848,Q3112659,False


# Some capitals do **not** have a start_date and an end_date. How many are?

In [ ]:
capitals_df

,wikidata_capital,capital_name,start_time,end_time,wikidata_region
0,Q82070,Tyre,NaN,NaN,Q41642
1,Q165841,Kar-Tukulti-Ninurta,NaN,NaN,Q41137
2,Q163490,Sidon,NaN,NaN,Q41642
3,Q199547,Harran,NaN,NaN,Q41137
4,Q200200,Assur,NaN,NaN,Q41137
...,...,...,...,...,...
1627,Q35775,Boise,NaN,NaN,Q1221
1628,Q37066,Bismarck,NaN,NaN,Q1207
1629,Q28198,Jackson,NaN,NaN,Q1494
1630,Q28848,Saint Paul,NaN,NaN,Q1527


In [ ]:
date_isna = capitals_df[capitals_df[['start_time', 'end_time']].isna().all(axis=1)]
date_isna

,wikidata_capital,capital_name,start_time,end_time,wikidata_region
0,Q82070,Tyre,NaN,NaN,Q41642
1,Q165841,Kar-Tukulti-Ninurta,NaN,NaN,Q41137
2,Q163490,Sidon,NaN,NaN,Q41642
3,Q199547,Harran,NaN,NaN,Q41137
4,Q200200,Assur,NaN,NaN,Q41137
...,...,...,...,...,...
1627,Q35775,Boise,NaN,NaN,Q1221
1628,Q37066,Bismarck,NaN,NaN,Q1207
1629,Q28198,Jackson,NaN,NaN,Q1494
1630,Q28848,Saint Paul,NaN,NaN,Q1527


# Some cities were capitals more than once. How many are?

In [ ]:
capitals_df

,wikidata_capital,capital_name,start_time,end_time,wikidata_region
0,Q82070,Tyre,NaN,NaN,Q41642
1,Q165841,Kar-Tukulti-Ninurta,NaN,NaN,Q41137
2,Q163490,Sidon,NaN,NaN,Q41642
3,Q199547,Harran,NaN,NaN,Q41137
4,Q200200,Assur,NaN,NaN,Q41137
...,...,...,...,...,...
1627,Q35775,Boise,NaN,NaN,Q1221
1628,Q37066,Bismarck,NaN,NaN,Q1207
1629,Q28198,Jackson,NaN,NaN,Q1494
1630,Q28848,Saint Paul,NaN,NaN,Q1527


In [ ]:
duplicated_rows = capitals_df.duplicated(subset=['wikidata_region', 'wikidata_capital'], keep=False)

In [ ]:
duplicated_capitals_df = capitals_df[duplicated_rows]
duplicated_capitals_df = duplicated_capitals_df.sort_values(by=['wikidata_region', 'wikidata_capital'])
duplicated_capitals_df

,wikidata_capital,capital_name,start_time,end_time,wikidata_region
825,Q172,Toronto,1856-01-01T00:00:00Z,1858-01-01T00:00:00Z,Q1121436
826,Q172,Toronto,1849-01-01T00:00:00Z,1852-01-01T00:00:00Z,Q1121436
824,Q2145,Quebec City,1852-01-01T00:00:00Z,1856-01-01T00:00:00Z,Q1121436
891,Q2145,Quebec City,1859-01-01T00:00:00Z,1866-01-01T00:00:00Z,Q1121436
376,Q187136,Luoyang,0759-01-01T00:00:00Z,0762-01-01T00:00:00Z,Q1188046
377,Q187136,Luoyang,0756-01-01T00:00:00Z,0757-01-01T00:00:00Z,Q1188046
327,Q1530,Baghdad,0766-01-01T00:00:00Z,0836-01-01T00:00:00Z,Q12536
328,Q1530,Baghdad,0892-01-01T00:00:00Z,1258-01-01T00:00:00Z,Q12536
325,Q1085,Prague,1346-01-01T00:00:00Z,1437-01-01T00:00:00Z,Q12548
326,Q1085,Prague,1583-01-01T00:00:00Z,1611-01-01T00:00:00Z,Q12548


In [ ]:
len(duplicated_capitals_df)

55

In [ ]:
distinct_capitals = duplicated_capitals_df['capital_name'].unique()
distinct_capitals_df = pd.DataFrame({'Capitals': distinct_capitals})
distinct_capitals_df

,Capitals
0,Toronto
1,Quebec City
2,Luoyang
3,Baghdad
4,Prague
5,Nanking
6,Chungking
7,Lahaina
8,Bago
9,Buda


# Exceptions - Checking redundandes

In [ ]:
# VERIFY CAPITALS THAT DID NOT GET A REGION_ID. WHAT TO DO?

value = ''
capitals_df.query('region_id == @value')

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
319,Q13364,Ravenna,0402-01-01T00:00:00Z,0476-01-01T00:00:00Z,Q2277,
320,Q16869,Constantinople,0476-01-01T00:00:00Z,1453-01-01T00:00:00Z,Q2277,
1366,Q1867,Taipei,1946-01-01T00:00:00Z,1956-01-01T00:00:00Z,Q32081,
1391,Q197930,Zhongxing New Village,1956-01-01T00:00:00Z,2018-07-01T00:00:00Z,Q32081,
333,Q85,Cairo,1261-01-01T00:00:00Z,1517-01-01T00:00:00Z,Q12536,


In [ ]:
value = 'Q19660'
capitals_df.query('wikidata_capital == @value')

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
901,Q19660,Bucharest,1881-01-01T00:00:00Z,1916-01-01T00:00:00Z,Q203493,1283
903,Q19660,Bucharest,1881-01-01T00:00:00Z,1947-01-01T00:00:00Z,Q203493,1283
902,Q19660,Bucharest,1918-01-01T00:00:00Z,1916-01-01T00:00:00Z,Q203493,1283
904,Q19660,Bucharest,1918-01-01T00:00:00Z,1947-01-01T00:00:00Z,Q203493,1283
525,Q19660,Bucharest,NaN,NaN,Q842794,"1284,1285,1286"
534,Q19660,Bucharest,NaN,NaN,Q958291,"1282,3016,3017"
659,Q19660,Bucharest,NaN,NaN,Q218,1287


In [ ]:
# VERIFY

value = 'Q2277'
capitals_df.query('wikidata_region == @value')

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
319,Q13364,Ravenna,0402-01-01T00:00:00Z,0476-01-01T00:00:00Z,Q2277,
320,Q16869,Constantinople,0476-01-01T00:00:00Z,1453-01-01T00:00:00Z,Q2277,
317,Q18287233,Roma,http://www.wikidata.org/.well-known/genid/b7a5...,0476-01-01T00:00:00Z,Q2277,"114,115"
321,Q220,Rome,-0026-01-01T00:00:00Z,0395-01-01T00:00:00Z,Q2277,"114,115"
318,Q490,Milan,0395-01-01T00:00:00Z,0402-01-01T00:00:00Z,Q2277,115


In [ ]:
# VERIFY

value = '???'
regions_df.query('start_date == @value')

,region_id,name,start_date,end_date,wikidata,has_capital
130,131,Middle Colonies,???,1776,Q6841282,False
131,132,Southern Colonies,???,1776,Q7569779,False


In [ ]:
# VERIFY

value = '???'
regions_df.query('end_date == @value')

,region_id,name,start_date,end_date,wikidata,has_capital
129,130,Kalhora dynasty,1701,???,Q6352367,False


In [ ]:
# VERIFY

value = 'Q19660'
capitals_df.query('wikidata_capital == @value')

,wikidata_capital,capital_name,start_time,end_time,wikidata_region,region_id
834,Q19660,Bucharest,1881-01-01T00:00:00Z,1916-01-01T00:00:00Z,Q203493,1283
836,Q19660,Bucharest,1881-01-01T00:00:00Z,1947-01-01T00:00:00Z,Q203493,1283
835,Q19660,Bucharest,1918-01-01T00:00:00Z,1916-01-01T00:00:00Z,Q203493,1283
837,Q19660,Bucharest,1918-01-01T00:00:00Z,1947-01-01T00:00:00Z,Q203493,1283
525,Q19660,Bucharest,NaN,NaN,Q842794,"1284,1285,1286"
534,Q19660,Bucharest,NaN,NaN,Q958291,"1282,3016,3017"
659,Q19660,Bucharest,NaN,NaN,Q218,1287


# Results Resume



*   **3252** regions on Region sheet
*   **7** did not have region_id
*   **22** do not have wikidata
*   from the **3223** regions left, **761** do not have capital registered (23.6%)
*   from the **1632** capitals found, **1262** do not have start and end date information.


**Questions**:

1.   There is one capital without wikidata uri, just with start_date and end_date (region Q467627). How should I present it?
2.   There are **24** cities that were capitals more than once. How could I add this information in the CSV file?


